In [ ]:
!pip install datasets transformers accelerate evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 1.6 MB/s eta 0:00:00


In [ ]:
import datasets
import torch

import evaluate

from torch.utils.data import DataLoader
from torch.utils.data.dataloader import default_collate
from transformers import AutoTokenizer, BertForSequenceClassification, AutoModelForSequenceClassification
from transformers import TrainingArguments, Trainer

In [ ]:
torch.manual_seed(123)

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [ ]:
  tokenizer = AutoTokenizer.from_pretrained("mrm8488/bert-mini-finetuned-age_news-classification")
  model = AutoModelForSequenceClassification.from_pretrained("mrm8488/bert-mini-finetuned-age_news-classification", num_labels=4)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
model = model.to(device)

In [ ]:
dataset = datasets.load_dataset('ag_news')

In [ ]:
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True, max_length=256, return_tensors='pt').to(device)

In [ ]:
train = dataset['train'].map(tokenize_function, batched=True)

Map:   0%|          | 0/120000 [00:00<?, ? examples/s]

In [ ]:
val = dataset['test'].map(tokenize_function, batched=True)

In [ ]:
training_args = TrainingArguments(
    output_dir="test_trainer",
    num_train_epochs=1,
    per_device_train_batch_size=64
    )

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train
)

In [ ]:
trainer.train()

Step,Training Loss
500,0.174500
1000,0.178000
1500,0.161600


Checkpoint destination directory test_trainer/checkpoint-500 already exists and is non-empty. Saving will proceed but saved results may be invalid.
Checkpoint destination directory test_trainer/checkpoint-1000 already exists and is non-empty. Saving will proceed but saved results may be invalid.
Checkpoint destination directory test_trainer/checkpoint-1500 already exists and is non-empty. Saving will proceed but saved results may be invalid.


TrainOutput(global_step=1875, training_loss=0.16999254353841145, metrics={'train_runtime': 333.701, 'train_samples_per_second': 359.603, 'train_steps_per_second': 5.619, 'total_flos': 594684887040000.0, 'train_loss': 0.16999254353841145, 'epoch': 1.0})

In [ ]:
class IMDbDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels, device='cuda'):
        self.encodings = encodings
        self.labels = labels
        self.device = device

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx]).to(self.device)
        return item

    def __len__(self):
        return len(self.labels)

In [ ]:
labels = val['label']
clear_val = val.map(remove_columns=['text', 'label'])
clear_val.set_format('pt')

In [ ]:
clear_val

Dataset({
    features: ['input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 7600
})

In [ ]:
val_loader = DataLoader(clear_val, batch_size=64)

In [ ]:
predict = []
for batch in val_loader:
  output = model(input_ids=batch['input_ids'].to(device),
                 token_type_ids=batch['token_type_ids'].to(device),
                 attention_mask=batch['attention_mask'].to(device))
  predict.extend(torch.argmax(output.logits, dim=1))

In [ ]:
acc = evaluate.load('accuracy')

In [ ]:
print(acc.compute(predictions=predict, references=labels))

{'accuracy': 0.9342105263157895}
